<a href="https://colab.research.google.com/github/yschoi9930/Python-for-Data-Analysis/blob/main/CHAPTER_8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84%ED%95%98%EA%B8%B0_%EC%A1%B0%EC%9D%B8%2C_%EB%B3%91%ED%95%A9%2C_%EB%B3%80%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 8. 데이터 준비하기 : 조인, 병합, 변형

## 8.1 계층적 색인
- 축에 대해 다중 색인 단계 지정 가능

In [ ]:
import pandas as pd
import numpy as np

- multiIndex를 색인으로 하는 Series

In [ ]:
data = pd.Series(np.random.randn(9), index = [['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

In [ ]:
data.index

- 부분적 색인으로 접근

In [ ]:
data['b']

In [ ]:
data['b':'c']

In [ ]:
data.loc[['b','d']]

In [ ]:
data.loc[:,2]

- 계층적인 색인은 데이터를 재형성하고 피벗테이블 생성 같은 그룹 기반의 작업을 할 때 중요하게 사용

In [ ]:
data

In [ ]:
data.unstack()

- 데이터 프레임은 두 축 모두 계층적 색인을 가질 수 있음음

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index= [['a','a','b','b'],[1,2,1,2]], columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame

- 계층적 색인의 각 단계는 이름을 가질 수 있음

In [ ]:
frame.index.names=['key1','key2']

In [ ]:
frame.columns.names = ['state', 'color']

In [ ]:
frame

In [ ]:
frame['Ohio']

In [ ]:
MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']], names=['state','color'])

### 계층의 순서를 바꾸고 정렬하기
- swaplevel은 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환함
- sort_index 메서드는 단일 계층에 속한 데이터를 정렬한다

In [ ]:
frame.swaplevel('key1','key2')

In [ ]:
frame

In [ ]:
frame.sort_index(level=0)

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame.swaplevel(0,1)

In [ ]:
# level= 0 -> key2 기준으로 정렬
frame.swaplevel(0,1).sort_index(level = 0)

In [ ]:
# level= 0 -> key1 기준으로 정렬
frame.swaplevel(0,1).sort_index(level = 1)

### 8.1.2 계층별 요약 통계계

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color', axis=1)

### 8.1.3 DataFrame 컬럼 사용하기

In [ ]:
frame = pd.DataFrame({'a':range(7), 'b' :range(7,0,-1), 'c' :['one','one','one','two','two','two', 'two'], 'd':[0,1,2,0,1,2,3]})
frame

- set_index : 하나 이상의 컬럼을 색인으로 하는 데이터프레임 생성

In [ ]:
frame.set_index(['c','d'])

In [ ]:
frame.set_index(['c','d'], drop =False)

In [ ]:
frame2 = frame.set_index(['c','d'])

- reset_indext : 계층적 색인 단계가 컬럼으로 이동

In [ ]:
frame2.reset_index()

## 8.2 데이터 합치기
- pd.merge : 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다.
- pd.concat : 하나의 축을 따라 객체를 이어붙인다
- combile_first : 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채운다.

### 8.2.1 데이터 베이스 스타일로 DataFrame 합치기
- merge, join

In [ ]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'], 'data1' : range(7)})
df2 = pd.DataFrame({'key':['a','b','d'], 'data2':range(3)})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1,df2)

- 어떤 컬럼을 병합할 것인지 명시하는 것이 좋음

In [ ]:
pd.merge(df1,df2, on = 'key')

In [ ]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'], 'data1' : range(7)})
df4 = pd.DataFrame({'rkey':['a','b','b'],'data2' : range(3)})

In [ ]:
df3

In [ ]:
df4

- 기준열 이름이 같을 때
pd.merge(left, right, on = '기준열', how = '조인방식')

- 기준열 이름이 다를 때
pd.merge(left, right, left_on = '왼쪽 열', right_on = '오른쪽 열', how = '조인방식')

In [ ]:
pd.merge(df3,df4, left_on = 'lkey', right_on ='rkey', how= 'outer')

- 여러 개의 키를 넘길 때

In [ ]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on = ['key1','key2'], how = 'outer')

In [ ]:
# 자동으로 공통 컬럼 찾아서 inner 조인해줌
pd.merge(left, right)

In [ ]:
# 자동으로 공통 컬럼인데 on에 안들어가면 이름 뒤에 붙여 구분해줌줌
pd.merge(left, right, on = 'key1')

In [ ]:
# 구분해주는 컬럼명을 지정하고 싶을 때
pd.merge(left, right, on = 'key1', suffixes = ('_left','_right'))

### 8.2.2 색인 병합하기

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [ ]:
left1

In [ ]:
right1

- 병합하려는 키가 한 테이블의 색인일 경우 : letf_index, right_index

In [ ]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

In [ ]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')

In [ ]:
lefth = pd.DataFrame({'data': [0,1,2,3,4], 'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'], 'key2' : [2000,2001,2002,2001,2002]})

In [ ]:
lefth

In [ ]:
righth = pd.DataFrame(np.arange(12).reshape((6,2)), columns = ['event1','event2'], index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],[2001,2000,2000,2000,2001,2002]])

In [ ]:
righth 